In [15]:
import numpy as np
import pandas as pd

In [16]:
df = pd.read_csv("dataset/TC-dataset.csv", sep="\t",index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 471910 entries, 0 to 541909
Data columns (total 8 columns):
BasketID           471910 non-null object
BasketDate         471910 non-null object
Sale               471910 non-null float64
CustomerID         406830 non-null float64
CustomerCountry    471910 non-null object
ProdID             471910 non-null object
ProdDescr          471157 non-null object
Qta                471910 non-null int64
dtypes: float64(2), int64(1), object(5)
memory usage: 32.4+ MB


## Gestione Customer Country

In [17]:
print(df.CustomerCountry.unique())
#Numero di righe con country Unspecified
print(len(df[df["CustomerCountry"].str.contains("Unspecified")]))

#Numero di righe con country European Community
print(len(df[df["CustomerCountry"].str.contains("European Community")]))

#Numero di righe con CustomerCountry Unspecified e CustomerID null
print(len(df[(df["CustomerCountry"].str.contains("Unspecified")) & (df["CustomerID"].isnull())]))

['United Kingdom' 'France' 'Australia' 'Netherlands' 'Germany' 'Norway'
 'EIRE' 'Switzerland' 'Spain' 'Poland' 'Portugal' 'Italy' 'Belgium'
 'Lithuania' 'Japan' 'Iceland' 'Channel Islands' 'Denmark' 'Cyprus'
 'Sweden' 'Austria' 'Israel' 'Finland' 'Bahrain' 'Greece' 'Hong Kong'
 'Singapore' 'Lebanon' 'United Arab Emirates' 'Saudi Arabia'
 'Czech Republic' 'Canada' 'Unspecified' 'Brazil' 'USA'
 'European Community' 'Malta' 'RSA']
340
61
96


In [18]:
count = 0

#Numero di carrelli il cui CustomerCountry risulta unspecified
print(len(df[df["CustomerCountry"].str.contains("Unspecified")].BasketID.unique()))

print()

                
#CustomerID che contengono una nazionalità unspecified
print(df[df["CustomerCountry"].str.contains("Unspecified")].CustomerID.unique())
print(len(df[df["CustomerCountry"].str.contains("Unspecified")].CustomerID.unique()))

print()

#Provo a vedere se tutti i record con quegli specifici CustomerID hanno la nazione per poterla recuperare
print(df[(df["CustomerID"].astype(str).str.contains('12743')) & (~df["CustomerCountry"].str.contains("Unspecified"))])

print()

#Record con Country Unspecified che non hanno il customer id a null raggruppati per numero di carrello
result = df[(df["CustomerCountry"].str.contains("Unspecified")) & (df["CustomerID"].notnull())]
result = result.groupby("BasketID").sum()
print(result)
print("Numero Elementi:" + str(len(result)))

print()

#Record con Country Unspecified che hanno il customer id a null raggruppati per numero di carrello
result = df[(df["CustomerCountry"].str.contains("Unspecified")) & (df["CustomerID"].isnull())]
result = result.groupby("BasketID").sum()
print(result)
print("Numero Elementi:" + str(len(result)))


print()
print(df['CustomerCountry'].mode()[0])



13

[12363. 16320.    nan 14265. 12743.]
5

Empty DataFrame
Columns: [BasketID, BasketDate, Sale, CustomerID, CustomerCountry, ProdID, ProdDescr, Qta]
Index: []

            Sale  CustomerID  Qta
BasketID                         
549687     36.50    197808.0  282
552695     96.81    767040.0  654
557499     42.10    146880.0   78
559929     51.91    213975.0  271
561658    283.06   1057669.0  205
561661    124.11    649893.0  114
563947     16.67     86541.0  126
564051    129.84    228240.0   59
Numero Elementi:8

           Sale  CustomerID  Qta
BasketID                        
553857    27.17         0.0   95
559521    58.56         0.0  207
565303    46.57         0.0  157
576646    27.80         0.0   88
578539    27.59         0.0  176
Numero Elementi:5

United Kingdom


### Considerazioni
Ci sono solo 4 customer con country unspecified e in totale hanno 244 record
Poi ci sono dei customerID null che hanno country unspecified e hanno 96 record

Dei 4 customer country ho controllato se fosse possibile recupere il Country in uno dei loro record, ma sono tutti Unspecified

Calcolando la moda dei dati è risultato che sia United Kingdom

Decisione: trasformare i 340 Unspecified in United Kingdom

## Record con CustomerID nulli 

In [7]:
len(df[df["CustomerID"].isnull()])

65080

Generiamo i nuovi customer ID

In [8]:
import random

basket_list_customer_null=df[df["CustomerID"].isnull()].BasketID.unique()
new_customer_per_basket_list=random.sample(range(1, 100000), len(basket_list_customer_null))


for i, elem in enumerate(basket_list_customer_null):
    df["CustomerID"][df.BasketID==elem]=str(new_customer_per_basket_list[i])+"N"

In [9]:
df[df["CustomerID"].isnull()]

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta


In [10]:
len(df[df["CustomerID"].astype(str).str.contains('N')])

65080

Sono stati generati nuovi CustomerId per non buttare transazioni mancanti

## Quantità negative con ordini non cancellati
Abbiamo già constatato che gli ordini con la C sono cancellati e abbiamo già deciso di cancellarli. Consideriamo ora quelli con solo le quantità negative. 

In [19]:
print(len(df[df["Qta"]<=0]))
print("Record già da cancellare: "+str(len(df[(df["Qta"]<=0) & (df["BasketID"].str.contains("C"))])))

print(len(df[(df["Qta"]<=0) & (df["BasketID"].str.contains("C")==False)]))

print(len(df[(df["Qta"]<=0) & (df["Sale"]<=0)&(df["BasketID"].str.contains("C")==False)]))

print("I record problematici sono 668, tutti quanti hanno anche Sale <=0\n")


9752
Record già da cancellare: 9084
668
668
I record problematici sono 668, tutti quanti hanno anche Sale <=0



Controlliamo la descrizione di questi prodotti:

In [20]:
print(df[(df["Qta"]<=0) & (df["BasketID"].str.contains("C")==False)].ProdDescr.unique())

[nan 'check' 'Dotcom sales' 'reverse 21/5/10 adjustment'
 'mouldy, thrown away.' '?' 'label mix up' 'samples/damages' 'thrown away'
 'damages' 'showroom' 'wrongly sold as sets' 'dotcom sold sets'
 'Amazon sold sets' 'wrongly sold sets' '?sold as sets?' 'damages/display'
 'damaged stock' 'damages?' 're dotcom quick fix.' 'sold in set?'
 'POSSIBLE DAMAGES OR LOST?' 'damaged' 'Damaged' 'DAMAGED' 'Missing'
 'wrong code?' 'crushed' 'damages/credits from ASOS.'
 'Not rcvd in 10/11/2010 delivery' 'Thrown away-rusty' 'damages/dotcom?'
 'smashed' 'reverse previous adjustment'
 'incorrectly credited C550456 see 47' 'wet damaged' 'Water damaged'
 'missing' 'sold as set on dotcom' 'mix up with c' 'ebay'
 'Sold as 1 on dotcom' 'taig adjust no stock' '?display?' '??'
 'OOPS ! adjustment' 'Dagamed' 'historic computer difference?....se'
 'incorrect stock entry.' 'wrongly coded-23343' 'stock check' 'WET/MOULDY'
 'Wet pallet-thrown away' 're-adjustment' 'wrongly coded 20713'
 'Unsaleable, destroyed.' 'd

Abbiamo solamente 668 transazioni in cui abbiamo la quantità negativa e non appartengono a ordini cancellati. Inoltre in questi 668 tutti hanno sale<=0.  Dalle descrizioni fornite la maggior parte sono quindi articoli persi oppure danneggiati o eventuali errori.
Verifichiamo però che non ci sia una trasazione speculare ma positiva. Proviamo con alcuni prodID

In [21]:
df[df["ProdID"]=="21414"]

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta
7192,537000,2010-12-03 15:32:00,0.0,NaN,United Kingdom,21414,NaN,-22
98375,544677,2011-02-22 16:13:00,2.1,18196.0,United Kingdom,21414,SCALLOP SHELL SOAP DISH,1
147790,549136,2011-04-06 15:23:00,0.0,NaN,United Kingdom,21414,NaN,-3


Non si rileva nessun particolare pattern che fa presumere la presenza di un corrispettivo ordine positivo.

*Decisione finale*: Eliminare anche questi sample a meno che non ci viene in mente qualche indicatore che possa tenere conto di questi articoli. In ogni caso sono solo 668 elementi.

## Sale <= 0 e quantità >0

In [24]:
#Numero di ordini con Sale < 0
print(len((df[df["Sale"]<=0])))
print()
print((df[df["Sale"]<0]))

a = df[(df["Qta"]>0) & (df["BasketID"].str.contains("C")==False)].ProdDescr.unique()
print(a)

1281

       BasketID           BasketDate      Sale  CustomerID CustomerCountry  \
299983  A563186  2011-08-12 14:51:00 -11062.06         NaN  United Kingdom   
299984  A563187  2011-08-12 14:52:00 -11062.06         NaN  United Kingdom   

       ProdID        ProdDescr  Qta  
299983      B  Adjust bad debt    1  
299984      B  Adjust bad debt    1  
['WHITE HANGING HEART T-LIGHT HOLDER' 'WHITE METAL LANTERN'
 'CREAM CUPID HEARTS COAT HANGER' ...
 'SET 10 CHRISTMAS CARDS HOHOHO 16956'
 'CREAM HANGING HEART T-LIGHT HOLDER' 'PAPER CRAFT , LITTLE BIRDIE']


In [31]:
#Numero di ordini con Sale = 0

print(len((df[df["Sale"]==0])))
print(len(df[(df["Sale"]==0) &(df["Qta"]==0)]))
print(len(df[(df["Sale"]==0) &(df["Qta"]>0)])) #Quantità positiva
print(len(df[(df["Sale"]==0) &(df["Qta"]<=0)])) #Quantità negativa
print(len(df[(df["Sale"]==0) &(df["Qta"]>0) & (df["CustomerID"].isnull())])) ##Customer null

1279
0
611
668
571


### Considerazione sui Sale
Esistono solo 2 record con sale < 0 a parer mio vanno assolutamente eliminati in quanto non rappresentano una informazione relativa ai customer ma sono relativi all'azienda
611 Record hanno sia il sale = 0 e la quantità > 0, essendo il nostro clustering fatto sui clienti e non sui prodotti credo che sia fuoriviante avere alcuni record con sale = 0
Pur giustificandoli come omaggi penso che la cosa più giusta sia eliminarli

Alcuni con il sale = 0 hanno quantità molto alte anche di 300 unità, in questo caso sarebbe anche difficile considerarli omaggi a meno che non siano per grossisti

Dato che stiamo performando una analisi sui customer trovo poco significativo considerare gli ordini "omaggio" che sono una registrazione che viene fatta dall'azienda per tenerne traccia (potrebbe deviare il clustering?)

Altra possibilità fare il clustering con queste Qta > 0 e vedere se effettivamente ci deviano il clustering, sono comunque pochi record.